# 注意此案例代码仅供学习参考，因tensorflow2.x版本问题无法运行！
# NOTE：Note that this case code is for learning reference only and cannot be run due to problems with the tensorflow2.x version!

In [ ]:
# 使用LeNet5的七层卷积神经网络用于MNIST手写数字识别 
# Using LeNet5's seven-layer convolutional neural network for MNIST handwritten digit recognition

import tensorflow as tf
import input_data
import tensorflow.compat.v1 as tf_v1


#读取训练数据到mnist变量里 Read training data into the mnist variable
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
tf_v1.disable_eager_execution()
# 为输入图像和目标输出类别创建节点 Create nodes for input images and target output categories
x = tf_v1.placeholder(tf_v1.float32, shape=[None, 784]) # 训练所需数据  占位符 Data required for training Placeholder
y_ = tf_v1.placeholder(tf_v1.float32, shape=[None, 10]) # 训练所需标签数据  占位符 Placeholder for label data required for training


# *************** 构建多层卷积网络 *************** #
# *************** Build a multi-layer convolutional network *************** #

# 权重、偏置、卷积及池化操作初始化,以避免在建立模型的时候反复做初始化操作 
# Initialize weights, biases, convolutions, and pooling operations to avoid repeated initialization when building the model

def weight_variable(shape):
  initial = tf_v1.compat.v1.random.truncated_normal(shape, stddev=0.1) # 取随机值，符合均值为0，标准差stddev为0.1 Take a random value with a mean of 0 and a standard deviation of 0.1
  return tf_v1.Variable(initial)

def bias_variable(shape):
  initial = tf_v1.constant(0.1, shape=shape)
  return tf_v1.Variable(initial)

# x 的第一个参数为图片的数量，第二、三个参数分别为图片高度和宽度，第四个参数为图片通道数。
# W 的前两个参数为卷积核尺寸，第三个参数为图像通道数，第四个参数为卷积核数量
# strides为卷积步长，其第一、四个参数必须为1，因为卷积层的步长只对矩阵的长和宽有效
# padding表示卷积的形式，即是否考虑边界。"SAME"是考虑边界，不足的时候用0去填充周围，"VALID"则不考虑
# The first parameter of x is the number of images, the second and third parameters are the image height and width, and the fourth parameter is the number of image channels.
# The first two parameters of W are the convolution kernel size, the third parameter is the number of image channels, and the fourth parameter is the number of convolution kernels
# strides is the convolution step size, and its first and fourth parameters must be 1, because the stride of the convolution layer is only valid for the length and width of the matrix
# padding indicates the form of convolution, that is, whether to consider the border. "SAME" considers the border, and fills the surrounding with 0 when it is insufficient, while "VALID" does not consider it
def conv2d(x, W):
  return tf_v1.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# x 参数的格式同tf.nn.conv2d中的x，ksize为池化层过滤器的尺度，strides为过滤器步长
# The format of the x parameter is the same as x in tf.nn.conv2d, ksize is the scale of the pooling layer filter, and strides is the filter step size
def max_pool_2x2(x):
  return tf_v1.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#把x更改为4维张量，第1维代表样本数量，第2维和第3维代表图像长宽， 第4维代表图像通道数 
#Change x to a 4-dimensional tensor, the first dimension represents the number of samples, the second and third dimensions represent the length and width of the image, and the fourth dimension represents the number of image channels
x_image = tf_v1.reshape(x, [-1,28,28,1]) # -1表示任意数量的样本数,大小为28x28，深度为1的张量 -1 represents any number of samples, a tensor of size 28x28 and depth 1

# 第一层：卷积 First layer: convolution
W_conv1 = weight_variable([5, 5, 1, 32]) # 卷积在每个5x5的patch中算出32个特征。 The convolution computes 32 features in each 5x5 patch.
b_conv1 = bias_variable([32])

h_conv1 = tf_v1.nn.relu(conv2d(x_image, W_conv1) + b_conv1) 

# 第二层：池化 Second layer: pooling
h_pool1 = max_pool_2x2(h_conv1)

# 第三层：卷积 The third layer: convolution
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf_v1.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

# 第四层：池化 Fourth layer: pooling
h_pool2 = max_pool_2x2(h_conv2)

# 第五层：全连接层 Layer 5: Fully connected layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf_v1.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf_v1.nn.relu(tf_v1.matmul(h_pool2_flat, W_fc1) + b_fc1)
 
# 在输出层之前加入dropout以减少过拟合 Add dropout before the output layer to reduce overfitting
keep_prob = tf_v1.placeholder("float")
h_fc1_drop = tf_v1.nn.dropout(h_fc1, keep_prob)

# 第六层：全连接层 Layer 6: Fully connected layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

# 第七层：输出层 Layer 7: Output layer
y_conv=tf_v1.nn.softmax(tf_v1.matmul(h_fc1_drop, W_fc2) + b_fc2)

# *************** 训练和评估模型 *************** #
# *************** Training and evaluating the model ****************** #

# 为训练过程指定最小化误差用的损失函数，即目标类别和预测类别之间的交叉熵
# Specify the loss function used to minimize the error for the training process, that is, the cross entropy between the target category and the predicted category
cross_entropy = -tf_v1.reduce_sum(y_*tf_v1.math.log(y_conv))

# 使用反向传播，利用优化器使损失函数最小化
# Use backpropagation to minimize the loss function using the optimizer
train_step = tf_v1.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# 检测我们的预测是否真实标签匹配(索引位置一样表示匹配)
# tf_v1.argmax(y_conv,dimension), 返回最大数值的下标 通常和tf.equal()一起使用，计算模型准确度
# dimension=0 按列找  dimension=1 按行找
# Check if our predictions match the actual labels (the same index position indicates a match)
# tf_v1.argmax(y_conv,dimension), returns the subscript of the maximum value. Usually used with tf.equal() to calculate model accuracy
# dimension=0 search by column dimension=1 search by row
correct_prediction = tf_v1.equal(tf_v1.argmax(y_conv,1), tf_v1.argmax(y_,1))  

# 统计测试准确率， 将correct_prediction的布尔值转换为浮点数来代表对、错，并取平均值。
# Statistical test accuracy, convert the Boolean value of correct_prediction to a floating point number to represent right and wrong, and take the average.
accuracy = tf_v1.reduce_mean(tf_v1.cast(correct_prediction, "float"))

saver = tf_v1.train.Saver() # 定义saver Defining saver

# *************** 开始训练模型 *************** #
# *************** Start training model ****************** #
with tf_v1.Session() as sess:
    sess.run(tf_v1.global_variables_initializer())

    for i in range(2000):
      batch = mnist.train.next_batch(25)
      if i%25 == 0:
        # 评估模型准确度，此阶段不使用Dropout Evaluate model accuracy. Dropout is not used at this stage.
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))

      # 训练模型，此阶段使用50%的Dropout Train the model, using 50% Dropout at this stage
      train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5}) 

    saver.save(sess, './model/model.ckpt') #模型储存位置 Model storage location

    print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images [0:2000], y_: mnist.test.labels [0:2000], keep_prob: 1.0}))
        
        

# 手写数字图像预处理  MNIST要求数据为28*28像素，单通道，且需要二值化。
# Handwritten digit image preprocessing MNIST requires the data to be 28*28 pixels, single channel, and needs to be binarized.

In [ ]:
'''
#此段针对图片不是黑白色或者大小不是28*28的时候的一个处理方法，下面图片直接是在画图软件28*28手动写的数字
import cv2
cut_img = cv2.imread("./images/4.png", 1)
resize_img = cv2.resize(cut_img, (28,28)) # 调整图像尺寸为28*28
ret, thresh_img = cv2.threshold(resize_img,127,255,cv2.THRESH_BINARY) # 二值
cv2.imwrite('./images/4_1.png', thresh_img)  # 预处理后图像保存位置

#This section is a processing method for images that are not black and white or are not 28*28 in size. The following image is a number written manually in 28*28 in a drawing software
import cv2
cut_img = cv2.imread("./images/4.png", 1)
resize_img = cv2.resize(cut_img, (28,28)) # Adjust the image size to 28*28
ret, thresh_img = cv2.threshold(resize_img,127,255,cv2.THRESH_BINARY) # Binary
cv2.imwrite('./images/4_1.png', thresh_img) # Save the image after preprocessing

'''

# 手写数字识别 Handwritten digit recognition

In [ ]:
from PIL import Image
import tensorflow as tf_v1
import tensorflow.compat.v1 as tf_v1
import numpy as np
import sys

im = Image.open('./images/3.png')
data = list(im.getdata())

result = [(255-x[0])*1.0/255.0 for x in data] 
print(result)



# 为输入图像和目标输出类别创建节点 Create nodes for input images and target output categories
x = tf_v1.placeholder("float", shape=[None, 784]) # 训练所需数据  占位符 Data required for training Placeholder

# *************** 构建多层卷积网络 *************** #
# *************** Build a multi-layer convolutional network *************** #
def weight_variable(shape):
  initial = tf_v1.compat.v1.random.truncated_normal(shape, stddev=0.1) # 取随机值，符合均值为0，标准差stddev为0.1 Take a random value with a mean of 0 and a standard deviation of 0.1
  return tf_v1.Variable(initial)

def bias_variable(shape):
  initial = tf_v1.constant(0.1, shape=shape)
  return tf_v1.Variable(initial)

def conv2d(x, W):
  return tf_v1.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf_v1.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x_image = tf_v1.reshape(x, [-1,28,28,1]) # -1表示任意数量的样本数,大小为28x28，深度为1的张量 -1 represents any number of samples, a tensor of size 28x28 and depth 1

W_conv1 = weight_variable([5, 5, 1, 32]) # 卷积在每个5x5的patch中算出32个特征。 The convolution computes 32 features in each 5x5 patch.
b_conv1 = bias_variable([32])
h_conv1 = tf_v1.nn.relu(conv2d(x_image, W_conv1) + b_conv1) 
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf_v1.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf_v1.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf_v1.nn.relu(tf_v1.matmul(h_pool2_flat, W_fc1) + b_fc1)

# 在输出层之前加入dropout以减少过拟合 Add dropout before the output layer to reduce overfitting
keep_prob = tf_v1.placeholder("float")
h_fc1_drop = tf_v1.nn.dropout(h_fc1, keep_prob)

# 全连接层 Fully connected layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

# 输出层 输出层
# tf_v1.nn.softmax()将神经网络的输层变成一个概率分布 Convert the input layer of the neural network into a probability distribution
y_conv=tf_v1.nn.softmax(tf_v1.matmul(h_fc1_drop, W_fc2) + b_fc2)

saver = tf_v1.train.Saver() # 定义saver Defining saver

# *************** 开始识别 *************** #
# *************** Start recognition *************** #
sess = tf_v1.Session()
sess.run(tf_v1.global_variables_initializer())
saver.restore(sess, "./model/model.ckpt")#这里使用了之前保存的模型参数 The previously saved model parameters are used here

prediction = tf_v1.argmax(y_conv,1)
predint = prediction.eval(feed_dict={x: [result],keep_prob: 1.0}, session=sess)

print("recognize result: %d" %predint[0])
sess.close()